In [1]:
pip install shared_utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import yaml
from shared_utils import portfolio_utils, schedule_rt_utils 
from segment_speed_utils import helpers
from calitp_data_analysis.sql import get_engine
db_engine = get_engine()
from gtfs_key_ntd_crosswalk import GCS_FILE_PATH, filter_to_valid_dates
from rapidfuzz import fuzz
import datetime as dt
import gcsfs
from calitp_data_analysis import get_fs
fs = get_fs()
import geopandas as gpd

OAuthError: ('Error code invalid_grant: Refresh token has expired', '{"error":"invalid_grant","error_description":"Refresh token has expired"}')

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def read_parquet_from_gcs(filename):
    gcs_path = GCS_FILE_PATH.replace("gs://", "") + filename
    with fs.open(gcs_path, 'rb') as f:
        return gpd.read_parquet(f)

In [ ]:
with open(
    "../_shared_utils/shared_utils/portfolio_organization_name.yml", "r"
) as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [ ]:
analysis_date_list = ['2022-11-30', '2022-12-03', '2022-12-04']

In [ ]:
# set date parameters
analysis_wkd = dt.date(2022,11,30)
analysis_sat = dt.date(2022,12,3)
analysis_sun = dt.date(2022,12,4)

In [ ]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [ ]:
new_operator_df = pd.concat(
    [helpers.import_scheduled_trips(
        one_date, 
        columns=["gtfs_dataset_key", "name", "trip_instance_key", "trip_id", "route_type", "shape_id", "route_id", "route_key", "direction_id", "route_short_name", "service_date"],
        get_pandas=True
    ) for one_date in analysis_date_list],
    ignore_index=True
)

In [ ]:
new_operator_df.head(5)

In [ ]:
new_operator_df.shape

In [ ]:
new_operator_df.schedule_gtfs_dataset_key.nunique(), new_operator_df.name.nunique()

In [ ]:
with db_engine.connect() as connection:
    query = """
        SELECT
            organization_source_record_id, schedule_gtfs_dataset_key, organization_key, schedule_gtfs_dataset_name, _valid_from, _valid_to
        FROM 
            cal-itp-data-infra.mart_transit_database.dim_provider_gtfs_data
    """
    dim_provider_gtfs_data = (
        pd.read_sql(query, connection)
        .pipe(schedule_rt_utils.localize_timestamp_col, ["_valid_from", "_valid_to"])
    )

In [ ]:
valid_provider_full = filter_to_valid_dates(dim_provider_gtfs_data, analysis_date_list)

In [ ]:
valid_provider_full.shape

In [ ]:
valid_provider_full = valid_provider_full.assign(
    portfolio_organization_name=valid_provider_full.schedule_gtfs_dataset_name.map(PORTFOLIO_ORGANIZATIONS_DICT)
)

In [ ]:
with db_engine.connect() as connection:
    query = """
        SELECT
            key, source_record_id, name, ntd_id_2022, ntd_agency_info_key, _is_current
        FROM 
            cal-itp-data-infra.mart_transit_database.dim_organizations
    """
    dim_orgs= pd.read_sql(query, connection)

In [ ]:
dim_orgs = dim_orgs[dim_orgs['_is_current'] == True]

In [ ]:
trips_provider_merge = pd.merge(
    new_operator_df.rename(columns={"name": "schedule_gtfs_dataset_name"}),
    valid_provider_full[["schedule_gtfs_dataset_key", "organization_source_record_id", "portfolio_organization_name" ]].drop_duplicates(),
    on="schedule_gtfs_dataset_key",
    how="left",
    indicator = True 
)

trips_provider_merge._merge.value_counts()


In [ ]:
trips_provider_merge = trips_provider_merge.drop(columns=['_merge'])
trips_gtfs_ntd_merge = pd.merge(
    trips_provider_merge,
    dim_orgs,
    left_on="organization_source_record_id",
    right_on="source_record_id",
    how="left",
    indicator=True
)

print(trips_gtfs_ntd_merge._merge.value_counts())

### GTFS Feed Key related to multiple NTD IDs

In [ ]:
trips_gtfs_ntd_merge.columns

In [ ]:
multiple_ntd_ids = (
    trips_gtfs_ntd_merge
    .groupby('schedule_gtfs_dataset_key')
    .agg(
        ntd_id_count=('ntd_id_2022', 'nunique'),
        organization_names=('name', lambda x: ', '.join(sorted(set(x.fillna('Not Available'))))),
        schedule_gtfs_dataset_name=('schedule_gtfs_dataset_name', lambda x: ', '.join(sorted(set(x.fillna('Not Available'))))),
        ntd_ids = ('ntd_id_2022', lambda x: ', '.join(sorted(set(x.fillna('Not Available')))))
    )
    .reset_index()
)

# Filter those with more than one unique ntd_id_2022
multiple_ntd_ids = multiple_ntd_ids[multiple_ntd_ids['ntd_id_count'] > 1]

multiple_ntd_ids.head()

Using fuzzy match technique to get just one ntd id, and one organization name 

In [ ]:
def get_orgs_and_ntd_ids(row, original_df, threshold=60):
    # Special case for 'VCTC GMV Schedule'
    if 'vctc gmv schedule' in row['schedule_gtfs_dataset_name'].lower():
        vctc_ntd_ids = original_df.loc[
            original_df['name'].str.contains('Ventura County Transportation Commission', case=False, na=False),
            'ntd_id_2022'
        ].dropna().astype(str).unique()
        return pd.Series({
            'organization_names': 'Ventura County Transportation Commission',
            'ntd_ids': ', '.join(sorted(vctc_ntd_ids))
        })
    
    schedule_name = row['schedule_gtfs_dataset_name'].lower()
    org_names = [x.strip() for x in row['organization_names'].split(',')]
    key = row['schedule_gtfs_dataset_key']

    matched_orgs = [
        org for org in org_names 
        if fuzz.partial_ratio(org.lower(), schedule_name) >= threshold
    ]
    
    if not matched_orgs:
        return pd.Series({
            'organization_names': row['organization_names'],
            'ntd_ids': row['ntd_ids']
        })

    filtered = original_df[
        (original_df['schedule_gtfs_dataset_key'] == key) &
        (original_df['name'].isin(matched_orgs))
    ]

    matched_ntd_ids = sorted(filtered['ntd_id_2022'].dropna().astype(str).unique())

    return pd.Series({
        'organization_names': ', '.join(matched_orgs),
        'ntd_ids': ', '.join(matched_ntd_ids)
    })

multiple_ntd_ids[['organization_names', 'ntd_ids']] = multiple_ntd_ids.apply(
    lambda row: get_orgs_and_ntd_ids(row, trips_gtfs_ntd_merge), axis=1
)

multiple_ntd_ids.head()

In [ ]:
update_df = multiple_ntd_ids[['schedule_gtfs_dataset_key', 'organization_names', 'ntd_ids']].rename(columns={'organization_names':'name', 'ntd_ids':'ntd_id_2022'})

trips_gtfs_ntd_merge = trips_gtfs_ntd_merge.merge(update_df, on='schedule_gtfs_dataset_key', how='left', suffixes=('', '_upd'))

trips_gtfs_ntd_merge['name'] = trips_gtfs_ntd_merge['name_upd'].combine_first(trips_gtfs_ntd_merge['name'])
trips_gtfs_ntd_merge['ntd_id_2022'] = trips_gtfs_ntd_merge['ntd_id_2022_upd'].combine_first(trips_gtfs_ntd_merge['ntd_id_2022'])

trips_gtfs_ntd_merge.drop(columns=['name_upd', 'ntd_id_2022_upd'], inplace=True)

### NTD organizations that are associated with more than one GTFS datasets

In [ ]:
multiple_gtfs_key = (
    trips_gtfs_ntd_merge
    .groupby('ntd_id_2022')
    .agg(
        gtfs_key_count=('schedule_gtfs_dataset_key', 'nunique'),
        organization_names=('name', lambda x: ', '.join(sorted(set(x.fillna('Not Available'))))),
        schedule_gtfs_dataset_name=('schedule_gtfs_dataset_name', lambda x: ', '.join(sorted(set(x.fillna('Not Available'))))),
        portfolio_organization_name=('portfolio_organization_name', lambda x: ', '.join(sorted(set(x.fillna('Not Available')))))
    )
    .reset_index()
)

# Filter those with more than one unique GTFS key
multiple_gtfs_key = multiple_gtfs_key[multiple_gtfs_key['gtfs_key_count'] > 1]

multiple_gtfs_key.head(20)

### Rows with missing ntd_id_2022 by GTFS feed key and name

In [ ]:
trips_gtfs_ntd_merge['ntd_id_2022'].isna().sum()

In [ ]:
missing_ntd_grouped = (
    trips_gtfs_ntd_merge[trips_gtfs_ntd_merge['ntd_id_2022'].isna()]
    .groupby(['schedule_gtfs_dataset_key', 'schedule_gtfs_dataset_name'])
    .agg(
        missing_ntd_count=('ntd_id_2022', 'size'),
        organization_names=('name', lambda x: ', '.join(sorted(set(x.fillna('Not Available')))))
    )
    .reset_index()
    .sort_values('missing_ntd_count', ascending=False)
)

missing_ntd_grouped['organization_names'].unique()

In [ ]:
trips_gtfs_ntd_merge = trips_gtfs_ntd_merge.dropna(subset=['ntd_id_2022'])

In [ ]:
trips_gtfs_ntd_merge.columns

In [ ]:
stoptimes_df = pd.concat(
    [helpers.import_scheduled_stop_times(
        one_date, 
        columns=["trip_id", "stop_id", "arrival_sec", "departure_sec"],
        get_pandas=True
    ) for one_date in analysis_date_list],
    ignore_index=True
)

In [ ]:
stops_geo = pd.concat(
    [helpers.import_scheduled_stops(
        one_date, 
        columns=["stop_id", "geometry", "stop_name"],
        get_pandas=True
    ) for one_date in analysis_date_list],
    ignore_index=True
)

In [ ]:
trips_gtfs_ntd_merge.shape

In [ ]:
def trips_data_summary(day_type, analysis_dt, trips_df, stoptimes_df, stops_geo):
    if trips_df.empty or stoptimes_df.empty or stops_geo.empty:
        return pd.DataFrame()

    # Add derived column
    trips_df["route_name_id"] = trips_df["route_short_name"].fillna("") + "_" + trips_df["route_id"]

    # Merge stoptimes_df with trips_df on trip_id to get full data
    merged = stoptimes_df.merge(trips_df, on="trip_id", how="left")

    # Group by gtfs_dataset_key, route_type, and stop_id
    grouped = merged.groupby(["schedule_gtfs_dataset_key", "route_type", "stop_id"])
    summary = grouped.agg(
        n_trips=("trip_instance_key", pd.Series.nunique),
        n_routes=("route_id", pd.Series.nunique),
        route_list=("route_name_id", lambda x: [x.tolist()])
    ).reset_index()

    summary["daytype"] = day_type
    summary["analysis_date"] = analysis_dt
    summary["route_list_string"] = summary["route_list"].apply(lambda x: ",".join(x[0]))

    # Merge additional info from trips_df using gtfs_dataset_key including portfolio_organization_name
    ntd_info = trips_df[[
        "schedule_gtfs_dataset_key", "ntd_id_2022", "ntd_agency_info_key", "service_date", "portfolio_organization_name"
    ]].drop_duplicates()
    summary = summary.merge(ntd_info, on="schedule_gtfs_dataset_key", how="left")

    # Merge with stops_geo on gtfs_dataset_key and stop_id
    stoptimes_geo = stops_geo.merge(summary, on=["stop_id"], how="left")

    return stoptimes_geo

In [ ]:
def stops_geo_acs_summary(stoptimes_geo, acs_ca, jobdata):
    if stoptimes_geo.empty:
        return pd.DataFrame()

    # Merge ACS with job data
    acs_ca = acs_ca.merge(jobdata, on='GEOID', how='left')
    acs_ca = acs_ca.to_crs(epsg=3347)

    # Spatial buffer (0.25 miles)
    stoptimes_geo = stoptimes_geo.to_crs(epsg=3347)
    stoptimes_geo["geometry"] = stoptimes_geo.buffer(402.336)

    # Spatial join with ACS
    stops_acs_joined = stoptimes_geo.sjoin(acs_ca, how="left", predicate="intersects")

    # Aggregate
    group_cols = [
        "gtfs_dataset_key", "ntd_id_2022", "ntd_agency_info_key", "stop_id", "stop_name", "n_trips", "n_routes",
        "daytype", "analysis_date", "route_list_string", "service_date"
    ]

    acs_summary = stops_acs_joined.groupby(group_cols).agg(
        sum_tracts=("GEOID", pd.Series.nunique),
        sum_total_pop=("total_pop", "sum"),
        sum_households=("households", "sum"),
        sum_not_us_citizen_pop=("not_us_citizen_pop", "sum"),
        sum_youth_pop=("youth_pop", "sum"),
        sum_seniors_pop=("seniors_pop", "sum"),
        sum_pop_determined_poverty_status=("pop_determined_poverty_status", "sum"),
        sum_poverty=("poverty", "sum"),
        sum_no_car=("workers_with_no_car", "sum"),
        sum_no_cars=("households_with_no_cars", "sum"),
        sum_land_area=("ALAND", "sum"),
        sum_jobs=("jobs_tot", "sum")
    ).reset_index()

    # Derived metrics
    acs_summary["land_area_sqkm"] = acs_summary["sum_land_area"] / 1_000_000
    acs_summary["pop_density"] = acs_summary["sum_total_pop"] / acs_summary["land_area_sqkm"]
    acs_summary["job_density"] = acs_summary["sum_jobs"] / acs_summary["land_area_sqkm"]
    acs_summary["pct_not_us_citizen_pop"] = (acs_summary["sum_not_us_citizen_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_youth_pop"] = (acs_summary["sum_youth_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_seniors_pop"] = (acs_summary["sum_seniors_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_poverty"] = (acs_summary["sum_poverty"] / acs_summary["sum_pop_determined_poverty_status"]) * 100
    acs_summary["pct_pop_workers_no_car"] = (acs_summary["sum_no_car"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_hh_no_cars"] = (acs_summary["sum_no_cars"] / acs_summary["sum_households"]) * 100

    return acs_summary

In [ ]:
acs_ca = read_parquet_from_gcs("tracts_ca_acs.parquet")
jobdata = pd.read_parquet(f"{GCS_FILE_PATH}/job_density_2022.parquet")[['GEOID', 'jobs_tot']]

In [ ]:
day_type_dates = [
    ("Weekday", analysis_wkd),
    ("Saturday", analysis_sat),
    ("Sunday", analysis_sun)
]

In [ ]:
import random

# Get unique GTFS keys
available_keys = trips_gtfs_ntd_merge["schedule_gtfs_dataset_key"].unique()

# Select one at random
target_gtfs_key = random.choice(available_keys)

print(f"Selected GTFS key: {target_gtfs_key}")

# Filter trips and stops for this key
trips_filtered = trips_gtfs_ntd_merge[trips_gtfs_ntd_merge["schedule_gtfs_dataset_key"] == target_gtfs_key]
stops_filtered = stops_geo[stops_geo["schedule_gtfs_dataset_key"] == target_gtfs_key]

# Loop through all day types
for day_type, analysis_dt in day_type_dates:
    print(f"\nProcessing {day_type} for GTFS key {target_gtfs_key}...")

    # Trips summary
    stoptimes_geo = trips_data_summary(
        day_type,
        analysis_dt,
        trips_filtered,
        stoptimes_df,
        stops_filtered
    )

    # ACS summary
    acs_summary = stops_geo_acs_summary(stoptimes_geo, acs_ca, jobdata)

    # Show result (or save/export)
    display(acs_summary.head())

    # Free memory
    del stoptimes_geo, acs_summary


Processing Weekday on 2022-11-30...


In [2]:
# import os
# os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

# import shared_utils
# import pandas as pd
# import geopandas as gpd
# from calitp_data_analysis.sql import get_engine
# from shared_utils import gtfs_utils_v2
# db_engine = get_engine()

# import gcsfs
# from calitp_data_analysis import get_fs
# from calitp_data_analysis import geography_utils, utils
# fs = get_fs()
# import re
# import google.auth
# import os
# credentials, project = google.auth.default()
# import pytz
# import datetime as dt

In [4]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [5]:
def read_parquet_from_gcs(filename):
    gcs_path = GCS_FILE_PATH.replace("gs://", "") + filename
    with fs.open(gcs_path, 'rb') as f:
        return gpd.read_parquet(f)

In [6]:
with db_engine.connect() as connection:
    query = """
        SELECT
            key, source_record_id, name, ntd_id_2022, ntd_agency_info_key, _is_current
        FROM 
            cal-itp-data-infra.mart_transit_database.dim_organizations
    """
    organizations= pd.read_sql(query, connection)

In [7]:
organizations = organizations[organizations['_is_current'] == True]
organizations = organizations.dropna(subset=["ntd_id_2022"])
organizations.shape

(245, 6)

In [9]:
has_duplicates = organizations['source_record_id'].duplicated().any()
print("Are there duplicates in 'source_record_id'? ->", has_duplicates)

Are there duplicates in 'source_record_id'? -> False


In [10]:
with db_engine.connect() as connection:
    query = """
        SELECT
            organization_source_record_id, schedule_gtfs_dataset_key, organization_key, schedule_gtfs_dataset_name, _valid_from, _valid_to
        FROM 
            cal-itp-data-infra.mart_transit_database.dim_provider_gtfs_data
    """
    dim_provider_gtfs_data= pd.read_sql(query, connection)

In [11]:
dim_provider_gtfs_data.shape

(29307, 6)

In [12]:
# dim_provider_gtfs_data['_valid_from'] = pd.to_datetime(dim_provider_gtfs_data['_valid_from']).dt.date
# dim_provider_gtfs_data['_valid_to'] = pd.to_datetime(dim_provider_gtfs_data['_valid_to']).dt.date

# Defininganalysis window
# analysis_start = dt.date(2022, 11, 30)
# analysis_end   = dt.date(2022, 12, 4)

#Filtering to only rows where the entire analysis period falls within valid range
# valid_gtfs_data = dim_provider_gtfs_data[
#     (dim_provider_gtfs_data['_valid_from'] <= analysis_start) &
#     (dim_provider_gtfs_data['_valid_to'] >= analysis_end)
# ]


In [13]:
dim_provider_gtfs_data.drop_duplicates(subset=['organization_source_record_id', 'schedule_gtfs_dataset_key'], inplace=True)

In [14]:
dim_provider_gtfs_data = dim_provider_gtfs_data.dropna(subset=['organization_source_record_id', 'schedule_gtfs_dataset_key'])

In [16]:
# Merge on organization_source_record_id and source_record_id
filtered_dimprovider_gtfs = dim_provider_gtfs_data.merge(
    organizations[['source_record_id', 'name', 'ntd_id_2022', 'ntd_agency_info_key']].drop_duplicates(), 
    left_on='organization_source_record_id', 
    right_on='source_record_id',
    how='inner'
)


In [17]:
filtered_dimprovider_gtfs.shape

(2868, 10)

In [18]:
filtered_dimprovider_gtfs.head(5)

,organization_source_record_id,schedule_gtfs_dataset_key,organization_key,schedule_gtfs_dataset_name,_valid_from,_valid_to,source_record_id,name,ntd_id_2022,ntd_agency_info_key
0,recOZgevYf7Jimm9L,a54ac3ee41ad2e04b8194f3d95f3003a,298ecce1563d88b1fb37298c8952cbd4,Bay Area 511 AC Transit Schedule,2022-08-20 00:00:00+00:00,2022-08-23 23:59:59.999999+00:00,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g
1,recOZgevYf7Jimm9L,ebf4ef573788dfc73a846a84459361f7,298ecce1563d88b1fb37298c8952cbd4,Bay Area 511 AC Transit Schedule,2022-09-03 00:00:00+00:00,2022-09-12 23:59:59.999999+00:00,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g
2,recOZgevYf7Jimm9L,52dac077979fbe9b69bdb1c43696706f,298ecce1563d88b1fb37298c8952cbd4,Bay Area 511 AC Transit Schedule,2022-08-10 00:00:00+00:00,2022-08-19 23:59:59.999999+00:00,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g
3,recOZgevYf7Jimm9L,d0db8366c430b3f407f54ebf70cb6f2b,298ecce1563d88b1fb37298c8952cbd4,Bay Area 511 AC Transit Schedule,2022-08-06 00:00:00+00:00,2022-08-09 23:59:59.999999+00:00,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g
4,recOZgevYf7Jimm9L,0c1737c9682679b777e0759d7540377a,6eb57c38e084b838a1c2f19614a74337,Bay Area 511 AC Transit Schedule,2022-10-26 00:00:00+00:00,2022-11-18 23:59:59.999999+00:00,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g


In [ ]:
analysis_date = ['2022-11-30', '2022-12-03', '2022-12-04']
trips = helpers.import_scheduled_trips(
        analysis_date,
        columns = ["gtfs_dataset_key", "feed_key", 
                   blah blah other columns you want,
        get_pandas = True,
    )

In [19]:
with db_engine.connect() as connection:
    query = """
        SELECT
            gtfs_dataset_key, feed_key, service_date, trip_instance_key, trip_id, route_id, shape_id, direction_id, route_type, route_short_name
        FROM 
            cal-itp-data-infra.mart_gtfs.fct_scheduled_trips
        WHERE 
            service_date IN (DATE '2022-11-30', DATE '2022-12-03', DATE '2022-12-04')
        ORDER BY service_date DESC
    """
    gtfs_trips= pd.read_sql(query, connection)

In [21]:
# Merge on organization_source_record_id and source_record_id
trips_ntd_data = gtfs_trips.merge(
    filtered_dimprovider_gtfs[['schedule_gtfs_dataset_key', 'name', 'ntd_id_2022', 'ntd_agency_info_key']], 
    left_on='gtfs_dataset_key', 
    right_on='schedule_gtfs_dataset_key',
    how='inner'
)

In [22]:
trips_ntd_data.shape

(1829843, 14)

In [31]:
trips_ntd_data[['gtfs_dataset_key', 'ntd_id_2022']].nunique()

gtfs_dataset_key    178
ntd_id_2022         146
dtype: int64

In [24]:
gtfs_counts = trips_ntd_data.groupby('gtfs_dataset_key')['ntd_id_2022'].nunique()
ntd_counts = trips_ntd_data.groupby('ntd_id_2022')['gtfs_dataset_key'].nunique()

# Filter to get only 1:1 mappings
valid_gtfs_keys = gtfs_counts[gtfs_counts == 1].index
valid_ntd_ids = ntd_counts[ntd_counts == 1].index

# Now filtering the original DataFrame
unique_pairs = trips_ntd_data[
    trips_ntd_data['gtfs_dataset_key'].isin(valid_gtfs_keys) &
    trips_ntd_data['ntd_id_2022'].isin(valid_ntd_ids)
]

In [32]:
unique_pairs[['gtfs_dataset_key', 'ntd_id_2022']].drop_duplicates().shape

(98, 2)

In [26]:
def trips_data_summary(day_type, analysis_dt, trips_df):
    if trips_df.empty:
        return pd.DataFrame()

    # Get feed_keys from the provided trips_df
    feed_keys = trips_df["feed_key"].unique().tolist()

    trips_df = trips_df.drop_duplicates()

    # Add derived column
    trips_df["route_name_id"] = trips_df["route_short_name"].fillna("") + "_" + trips_df["route_id"]

    # Get stop_times
    stoptimes_df = gtfs_utils_v2.get_stop_times(
        selected_date=analysis_dt,
        operator_feeds=feed_keys,
        stop_time_cols=["trip_id", "feed_key", "stop_id", "arrival_sec", "departure_sec"],
        trip_df=trips_df,
        get_df=True
    )

    if stoptimes_df.empty:
        return pd.DataFrame()

    # Merge with trips
    stoptimes_df = stoptimes_df.merge(trips_df, on=["trip_id", "feed_key"])

    # Summarize stop-level trips
    grouped = stoptimes_df.groupby(["feed_key", "route_type", "stop_id"])
    summary = grouped.agg(
        n_trips=("trip_instance_key", pd.Series.nunique),
        n_routes=("route_id", pd.Series.nunique),
        route_list=("route_name_id", lambda x: [x.tolist()])
    ).reset_index()

    summary["daytype"] = day_type
    summary["analysis_date"] = analysis_dt
    summary["route_list_string"] = summary["route_list"].apply(lambda x: ",".join(x[0]))

    # Add gtfs_dataset_key and service_date from trips_df to summary
    feed_info = trips_df[["feed_key", "ntd_id_2022", "ntd_agency_info_key", "gtfs_dataset_key", "service_date"]].drop_duplicates()
    summary = summary.merge(feed_info, on="feed_key", how="left")

    # Get stop metadata
    stops_geo = gtfs_utils_v2.get_stops(
        selected_date=analysis_dt,
        operator_feeds=feed_keys,
        stop_cols=["feed_key", "stop_id", "stop_desc", "stop_name"],
        get_df=True
    )

    # Join with summarized data
    stoptimes_geo = stops_geo.merge(summary, on=["feed_key", "stop_id"])

    return stoptimes_geo


In [27]:
def stops_geo_acs_summary(stoptimes_geo, acs_ca, jobdata):
    if stoptimes_geo.empty:
        return pd.DataFrame()

    # Merge ACS with job data
    acs_ca = acs_ca.merge(jobdata, on='GEOID', how='left')
    acs_ca = acs_ca.to_crs(epsg=3347)

    # Spatial buffer (0.25 miles)
    stoptimes_geo = stoptimes_geo.to_crs(epsg=3347)
    stoptimes_geo["geometry"] = stoptimes_geo.buffer(402.336)

    # Spatial join with ACS
    stops_acs_joined = stoptimes_geo.sjoin(acs_ca, how="left", predicate="intersects")

    # Aggregate
    group_cols = [
        "gtfs_dataset_key", "ntd_id_2022", "ntd_agency_info_key", "stop_id", "stop_name", "n_trips", "n_routes",
        "daytype", "analysis_date", "route_list_string", "service_date"
    ]

    acs_summary = stops_acs_joined.groupby(group_cols).agg(
        sum_tracts=("GEOID", pd.Series.nunique),
        sum_total_pop=("total_pop", "sum"),
        sum_households=("households", "sum"),
        sum_not_us_citizen_pop=("not_us_citizen_pop", "sum"),
        sum_youth_pop=("youth_pop", "sum"),
        sum_seniors_pop=("seniors_pop", "sum"),
        sum_pop_determined_poverty_status=("pop_determined_poverty_status", "sum"),
        sum_poverty=("poverty", "sum"),
        sum_no_car=("workers_with_no_car", "sum"),
        sum_no_cars=("households_with_no_cars", "sum"),
        sum_land_area=("ALAND", "sum"),
        sum_jobs=("jobs_tot", "sum")
    ).reset_index()

    # Derived metrics
    acs_summary["land_area_sqkm"] = acs_summary["sum_land_area"] / 1_000_000
    acs_summary["pop_density"] = acs_summary["sum_total_pop"] / acs_summary["land_area_sqkm"]
    acs_summary["job_density"] = acs_summary["sum_jobs"] / acs_summary["land_area_sqkm"]
    acs_summary["pct_not_us_citizen_pop"] = (acs_summary["sum_not_us_citizen_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_youth_pop"] = (acs_summary["sum_youth_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_seniors_pop"] = (acs_summary["sum_seniors_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_poverty"] = (acs_summary["sum_poverty"] / acs_summary["sum_pop_determined_poverty_status"]) * 100
    acs_summary["pct_pop_workers_no_car"] = (acs_summary["sum_no_car"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_hh_no_cars"] = (acs_summary["sum_no_cars"] / acs_summary["sum_households"]) * 100

    return acs_summary


In [28]:
# set date parameters
analysis_wkd = dt.date(2022,11,30)
analysis_sat = dt.date(2022,12,3)
analysis_sun = dt.date(2022,12,4)

In [29]:
acs_ca = read_parquet_from_gcs("tracts_ca_acs.parquet")
jobdata = pd.read_parquet(f"{GCS_FILE_PATH}/job_density_2022.parquet")[['GEOID', 'jobs_tot']]

In [30]:
# List of (day_type, analysis_date) pairs
day_type_dates = [
    ("Weekday", analysis_wkd),
    ("Saturday", analysis_sat),
    ("Sunday", analysis_sun)
]
